In [2]:
!pip install xarray netCDF4 folium branca geopandas rasterio rioxarray pyproj shapely matplotlib seaborn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 41.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.5/32.5 MB 45.9 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: clickm━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/11 [pyogrio]
    Found existing installation: click 8.2.1━━━━━━━━━━━━━━━━━━  1/11 [pyogrio]
    Uninstalling click-8.2.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/11 [pyogrio]
      Successfully uninstalled click-8.2.1━━━━━━━━━━━━━━━━━━━━  1/11 [pyogrio]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [rioxarray]11 [folium]as]


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree


# ----------------------------
# 파일 경로 (사용자 업로드 경로)
# ----------------------------
OCEAN_CSV = "../Dataset/ocean_grid_points_0p25deg.csv"          # dataset 1
WIND10_NC = "../Dataset/netcdf4_korea_10m_wind.nc"             # dataset 2 (u10, v10)
WIND100_NC = "../Dataset/netcdf4_korea_100m_wind.nc"           # dataset 2 (u100, v100)
OPER_NC = "../Dataset/data_stream-oper_stepType-instant.nc"    # dataset 3 (sst 등)
WAVE_NC = "../Dataset/data_stream-wave_stepType-instant.nc"    # dataset 3 (rhoao)

# ----------------------------
# 유틸: 해양 마스크 생성
# - dataset1 포인트(위경도 리스트)를 기준으로,
#   타겟 격자(lat/lon)에서 해양 여부(가까운 포인트 존재)를 True/False로 생성
# ----------------------------
def build_ocean_mask(lat_1d: np.ndarray, lon_1d: np.ndarray, ocean_df: pd.DataFrame,
                     max_dist_deg: float = 0.15) -> xr.DataArray:
    """
    lat_1d, lon_1d: 타겟 격자의 1D 위도/경도 좌표
    ocean_df: columns=['lat','lon']인 해양 포인트 데이터프레임                                                                                                                       
    max_dist_deg: 최근접 해양 포인트까지의 허용 최대 거리(도 단위)
                  - 0.25도 격자면 0.15 정도가 보통 안전(최근접 포인트 없으면 육지로 간주)
    반환: (latitude, longitude) 차원의 boolean DataArray
    """
    # KDTree는 (lon, lat) 또는 (lat, lon) 중 하나로 통일해서 사용
    ocean_pts = np.column_stack([ocean_df["lat"].to_numpy(), ocean_df["lon"].to_numpy()])
    tree = cKDTree(ocean_pts)

    LAT, LON = np.meshgrid(lat_1d, lon_1d, indexing="ij")  # (lat, lon)
    target_pts = np.column_stack([LAT.ravel(), LON.ravel()])

    dist, _ = tree.query(target_pts, k=1)
    mask = (dist <= max_dist_deg).reshape(LAT.shape)

    return xr.DataArray(
        mask,
        coords={"latitude": lat_1d, "longitude": lon_1d},
        dims=("latitude", "longitude"),
        name="ocean_mask"
    )


# ----------------------------
# 유틸: 풍속/풍향 계산
# - u: 동서 성분(동쪽 +), v: 남북 성분(북쪽 +)
# - 풍속: sqrt(u^2 + v^2)
# - 풍향(From, meteorological): 바람이 "불어오는" 방향(북=0°, 동=90°)
# ----------------------------
def wind_speed_dir_from(u: xr.DataArray, v: xr.DataArray): 
    speed = np.hypot(u, v)
    # meteorological "from" direction
    wdir = (np.degrees(np.arctan2(-u, -v)) + 360.0) % 360.0
    wdir = xr.DataArray(wdir, coords=u.coords, dims=u.dims, name="wind_dir_from_deg")
    speed = xr.DataArray(speed, coords=u.coords, dims=u.dims, name="wind_speed")
    return speed, wdir


# ----------------------------
# 유틸: rho(공기밀도) 타겟 격자/시간에 맞춰 보간
# - wave의 rhoao는 (valid_time, latitude, longitude)이며 해상도(0.5°)가 더 성김
# ----------------------------
def regrid_rho_to_target(ds_wave: xr.Dataset, target_da: xr.DataArray) -> xr.DataArray:
    rho = ds_wave["rhoao"]
    # target_da의 좌표(시간/위도/경도)에 맞춰 보간
    # 시간/공간 모두 기본 linear 보간 (필요 시 method='nearest'로 변경 가능)
    rho_i = rho.interp(
        valid_time=target_da["valid_time"],
        latitude=target_da["latitude"],
        longitude=target_da["longitude"],
    )
    rho_i.name = "rho_air"
    return rho_i

      
# ----------------------------
# 플롯 유틸 (Cartopy 없이 위경도 평면으로 그립니다)
# ----------------------------
def plot_speed_and_vectors(speed2d, u2d, v2d, title, out_png,
                           stride=3, vmin=None, vmax=None):
    lat = speed2d["latitude"].to_numpy()
    lon = speed2d["longitude"].to_numpy()
    LON, LAT = np.meshgrid(lon, lat)

    fig, ax = plt.subplots(figsize=(10, 7))
    pm = ax.pcolormesh(LON, LAT, speed2d.to_numpy(), shading="auto", vmin=vmin, vmax=vmax)
    cb = fig.colorbar(pm, ax=ax)
    cb.set_label("Wind speed (m/s)")

    # 벡터는 너무 촘촘하면 보기 어려우니 subsampling
    ax.quiver(
        LON[::stride, ::stride], LAT[::stride, ::stride],
        u2d.to_numpy()[::stride, ::stride], v2d.to_numpy()[::stride, ::stride],
        scale=400, width=0.002
    )

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    ax.grid(True, linewidth=0.5, alpha=0.5)

    fig.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)


def plot_direction_map(wdir2d, title, out_png):
    lat = wdir2d["latitude"].to_numpy()
    lon = wdir2d["longitude"].to_numpy()
    LON, LAT = np.meshgrid(lon, lat)

    fig, ax = plt.subplots(figsize=(10, 7))
    pm = ax.pcolormesh(LON, LAT, wdir2d.to_numpy(), shading="auto", vmin=0, vmax=360, cmap="hsv")
    cb = fig.colorbar(pm, ax=ax)
    cb.set_label("Wind direction (from, deg)")

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    ax.grid(True, linewidth=0.5, alpha=0.5)

    fig.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)


def plot_power_density(power2d, title, out_png, vmin=None, vmax=None):
    lat = power2d["latitude"].to_numpy()
    lon = power2d["longitude"].to_numpy()
    LON, LAT = np.meshgrid(lon, lat)

    fig, ax = plt.subplots(figsize=(10, 7))
    pm = ax.pcolormesh(LON, LAT, power2d.to_numpy(), shading="auto", vmin=vmin, vmax=vmax)
    cb = fig.colorbar(pm, ax=ax)
    cb.set_label("Wind power density (W/m²)")

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    ax.grid(True, linewidth=0.5, alpha=0.5)

    fig.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)


# ----------------------------
# 메인 분석/맵 생성
# ----------------------------
def main():
    # 1) 데이터 로드
    ocean_df = pd.read_csv(OCEAN_CSV)  # columns: lat, lon

    ds10 = xr.open_dataset(WIND10_NC)     # u10, v10
    ds100 = xr.open_dataset(WIND100_NC)   # u100, v100
    ds_wave = xr.open_dataset(WAVE_NC)    # rhoao (kg/m^3)

    # 2) 분석할 시간 구간 선택
    t0, t1 = "2024-01-01T00:00:00", "2024-12-31T23:00:00"

    # 원하면 특정 시각 스냅샷으로 변경:
    # ts = "2024-07-15T12:00:00"
    # da_u100 = ds100["u100"].sel(valid_time=ts)
    # da_v100 = ds100["v100"].sel(valid_time=ts)

    da_u10 = ds10["u10"].sel(valid_time=slice(t0, t1)).mean("valid_time")
    da_v10 = ds10["v10"].sel(valid_time=slice(t0, t1)).mean("valid_time")

    da_u100 = ds100["u100"].sel(valid_time=slice(t0, t1)).mean("valid_time")
    da_v100 = ds100["v100"].sel(valid_time=slice(t0, t1)).mean("valid_time")

    # 평균을 내면 valid_time 차원이 사라지므로, rho 보간을 위해 임시로 valid_time 유지 버전도 만들 수 있습니다.
    # 풍력 밀도에서 rho를 시간 평균으로 쓰고 싶으면 아래처럼 처리합니다.
    rho_time_mean = ds_wave["rhoao"].sel(valid_time=slice(t0, t1)).mean("valid_time")

    # 3) 해양 마스크 생성 (dataset1 기반)
    lat = ds100["latitude"].to_numpy()
    lon = ds100["longitude"].to_numpy()
    ocean_mask = build_ocean_mask(lat, lon, ocean_df, max_dist_deg=0.15)

    # 4) 풍속/풍향 계산 (10m, 100m)
    spd10, dir10 = wind_speed_dir_from(da_u10, da_v10)
    spd100, dir100 = wind_speed_dir_from(da_u100, da_v100)

    # 5) 풍력 에너지(풍력 밀도) 계산: P = 0.5 * rho * V^3
    # - rho는 dataset3 wave의 rhoao(kg/m^3)를 사용(시간 평균 및 격자 보간 필요 시 처리)
    # - ds_wave는 0.5° 격자, ds100은 0.25° 격자 -> 공간 보간
    rho_on_025 = rho_time_mean.interp(latitude=ds100["latitude"], longitude=ds100["longitude"])

    power100 = 0.5 * rho_on_025 * (spd100 ** 3)
    power100.name = "wind_power_density_100m"

    # 6) 해양 영역만 남기기 (육지는 NaN)
    spd10_o = spd10.where(ocean_mask)
    dir10_o = dir10.where(ocean_mask)
    spd100_o = spd100.where(ocean_mask)
    dir100_o = dir100.where(ocean_mask)
    power100_o = power100.where(ocean_mask)

    # 7) 지도 저장 (예: 100m 기준)
    plot_speed_and_vectors(
        speed2d=spd100_o,
        u2d=da_u100.where(ocean_mask),
        v2d=da_v100.where(ocean_mask),
        title=f"Wind speed (100m) + vectors | Mean {t0} to {t1} (Ocean only)",
        out_png="wind_speed_100m_ocean.png",
        stride=3
    )

    plot_direction_map(
        wdir2d=dir100_o,
        title=f"Wind direction (from, 100m) | Mean {t0} to {t1} (Ocean only)",
        out_png="wind_dir_100m_ocean.png"
    )

    plot_power_density(
        power2d=power100_o,
        title=f"Wind power density (100m) | Mean {t0} to {t1} (Ocean only)",
        out_png="wind_power_density_100m_ocean.png"
    )

    # 10m도 필요하면 동일하게 저장
    plot_speed_and_vectors(
        speed2d=spd10_o,
        u2d=da_u10.where(ocean_mask),
        v2d=da_v10.where(ocean_mask),
        title=f"Wind speed (10m) + vectors | Mean {t0} to {t1} (Ocean only)",
        out_png="wind_speed_10m_ocean.png",
        stride=3
    )

    plot_direction_map(
        wdir2d=dir10_o,
        title=f"Wind direction (from, 10m) | Mean {t0} to {t1} (Ocean only)",
        out_png="wind_dir_10m_ocean.png"
    )

    print("Done. Saved PNG files:")
    print(" - wind_speed_100m_ocean.png")
    print(" - wind_dir_100m_ocean.png")
    print(" - wind_power_density_100m_ocean.png")
    print(" - wind_speed_10m_ocean.png")
    print(" - wind_dir_10m_ocean.png")


if __name__ == "__main__":
    main()


Done. Saved PNG files:
 - wind_speed_100m_ocean.png
 - wind_dir_100m_ocean.png
 - wind_power_density_100m_ocean.png
 - wind_speed_10m_ocean.png
 - wind_dir_10m_ocean.png
